In [34]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn

In [23]:
x_data = [[1,2],[2,3],[3,1],
        [4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]] 

In [24]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

## w, b 데이터 지정
- x_train : 6x2 * 2x1(weights) ==> 6x1 + b (bias: 1)
- sigmoid(wx+b) ==> if w==0 and b==0 ==> sigmoid(0)  ==> 0.5

In [25]:
w = torch.zeros((2,1), requires_grad=True)
# w = torch.randn((2,1), requires_grad=True)  # 이렇게 해도 됨
b = torch.zeros(1, requires_grad=True)

In [26]:
optimizer = optim.SGD([w,b], lr=1)

In [28]:
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    # 1. Hypothesis 함수 : Forward
    # sigmoid function 작성한 것을 import 해서  사용해도 됨
    # sigmoid = 1/1+e^(-wx+b) 와 같음
    hx = 1 / (1+ torch.exp(-(x_train.matmul(w) + b)))
    
    # 2. Cost Function 사용
    # cost(hx) 1 : -log(h(x))
    # cost(hx) 0 : -log(1-h(x))
    cost = -((y_train * torch.log(hx)) + ((1 - y_train) * torch.log(1-hx))).mean()
    # cost = F.binary_cross_entorpy(hx, y_train) 와 동일
    
    # 3. 최적화 (loss, cost function을 미분하고, 
    # 경사타고 내려오면서 w, b 업데이트)
    optimizer.zero_grad()  # 누적 방지
    cost.backward()  # loss function 미분
    optimizer.step()  # SGD 작동 : lr 만큼 내려가면서 w,b 업데이트
    
    # 4. loss 값 출력
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs}, cost: {cost.item():.6f}")
    
print(f"w : {w}\nb : {b}")    

Epoch    0/1000, cost: 0.019834
Epoch  100/1000, cost: 0.018149
Epoch  200/1000, cost: 0.016730
Epoch  300/1000, cost: 0.015517
Epoch  400/1000, cost: 0.014469
Epoch  500/1000, cost: 0.013554
Epoch  600/1000, cost: 0.012748
Epoch  700/1000, cost: 0.012033
Epoch  800/1000, cost: 0.011394
Epoch  900/1000, cost: 0.010819
Epoch 1000/1000, cost: 0.010300
w : tensor([[3.8961],
        [1.8565]], requires_grad=True)
b : tensor([-17.4290], requires_grad=True)


# inference
- model parameter를 이용한 inference
- training dataset에 대해서 inference

In [29]:
hypothesis = torch.sigmoid(x_train.matmul(w)+b)
hypothesis

tensor([[5.4359e-05],
        [1.6836e-02],
        [2.0150e-02],
        [9.7645e-01],
        [9.9951e-01],
        [9.9994e-01]], grad_fn=<SigmoidBackward0>)

## sigmoid의 출력값이 0.5 이상이면 1, 아니면 0으로 출력

In [30]:
pred = []
for i in list(hypothesis):
    if i >= 0.5:
        pred.append(1)
    else: 
        pred.append(0)
pred

[0, 0, 0, 1, 1, 1]

In [31]:
# 위 셀을 아래와 같이도 가능함
prediction = hypothesis >= torch.FloatTensor([0.5])
prediction

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

---
# pytorch로 구현하기
```python
nn.Sequential()  # nn.Module 층을 차례로 쌓을 수 있도록 함
```

In [35]:
model = nn.Sequential(
    nn.Linear(2, 1),  # input Dim 2, output Dim 1
    nn.Sigmoid()  # 출력은 시그모이드 결과
)

In [36]:
optimizer = optim.SGD(model.parameters(), lr=1)

In [42]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        
        # 실제 y_train 값(GT)과 예측된 값을 비교한 결과
        correct_pred = prediction == y_train
        accuracy = correct_pred.sum().item() / len(correct_pred)  # True=1이므로 맞춘 개수
        # correct_pred.sum() 하면 tensor(6) 형태로 나오기 때문에 그 값(6)만 가져오기 위해 item() 붙여줌
        
        print(f"Epoch {epoch:4d}/{nb_epochs}, cost: {cost.item():.6f},"
              f" Accuracy: {accuracy * 100: 3.2f}%")

Epoch    0/1000, cost: 0.006956, Accuracy:  100.00%
Epoch  200/1000, cost: 0.006533, Accuracy:  100.00%
Epoch  400/1000, cost: 0.006159, Accuracy:  100.00%
Epoch  600/1000, cost: 0.005825, Accuracy:  100.00%
Epoch  800/1000, cost: 0.005526, Accuracy:  100.00%
Epoch 1000/1000, cost: 0.005256, Accuracy:  100.00%
